In [2]:
import scanpy as sc
import pandas as pd
import numpy as np

GSE239626_PATH = 'GSE239626'
GSE138266_PATH = 'GSE138266'
GSE194078_PATH = 'GSE194078'

GSE239626_LABELS_PATH = 'results/celltypist_labels_from_protein_Immune_All_High.csv'
GSE138266_LABELS_PATH = 'results/celltypist_labels_from_transcriptomic_GSE138266.csv'
GSE194078_LABELS_PATH = 'results/celltypist_labels_from_transcriptomic_GSE194078.csv'

c:\Users\giuli\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### GSE239626

In [3]:
adata_GSE239626 = sc.read(GSE239626_PATH)

GSE239626_labels = pd.read_csv(GSE239626_LABELS_PATH)
adata_GSE239626.obs['cell_types_labels'] = GSE239626_labels['predicted_labels'].values

adata_GSE239626

AnnData object with n_obs × n_vars = 72317 × 36601
    obs: 'dataset', 'disease', 'cell_type', 'patiens', 'patiens_id', 'time', 'cell_types_labels'
    var: 'gene_ids', 'feature_types'

In [4]:
# rename columns 'patiens' to 'patient' and 'patiens_id' to 'patient_id'
adata_GSE239626.obs.rename(columns={'patiens': 'sample', 'patiens_id': 'patient_id'}, inplace=True)

In [5]:
adata_GSE239626.obs

,dataset,disease,cell_type,sample,patient_id,time,cell_types_labels
AAACCCAAGACTGTTC-1,GSE239626,MS,PBMC,GSM7669046,N1,J0,T cells
AAACCCAAGGATCATA-1,GSE239626,MS,PBMC,GSM7669046,N1,J0,T cells
AAACCCAGTTATTCTC-1,GSE239626,MS,PBMC,GSM7669046,N1,J0,B cells
AAACCCATCATGAGGG-1,GSE239626,MS,PBMC,GSM7669046,N1,J0,T cells
AAACGAAAGCCAGTAG-1,GSE239626,MS,PBMC,GSM7669046,N1,J0,B cells
...,...,...,...,...,...,...,...
TTTGTTGAGCGTCGAA-20,GSE239626,MS,PBMC,GSM7669084,N5,M3,T cells
TTTGTTGAGTGTACCT-20,GSE239626,MS,PBMC,GSM7669084,N5,M3,T cells
TTTGTTGCAATTGCAC-20,GSE239626,MS,PBMC,GSM7669084,N5,M3,T cells
TTTGTTGCATTCAGCA-20,GSE239626,MS,PBMC,GSM7669084,N5,M3,T cells


### GSE138266

In [6]:
adata_GSE138266 = sc.read(GSE138266_PATH)
adata_GSE138266 = adata_GSE138266[(adata_GSE138266.obs['cell_type']!='CSF')|(adata_GSE138266.obs['disease']!='PST')]
adata_GSE138266

View of AnnData object with n_obs × n_vars = 68038 × 33694
    obs: 'sample', 'patient', 'dataset', 'disease', 'cell_type', 'batch'
    var: 'gene_symbols', 'gene_ids'

In [7]:
adata_GSE138266.obs.head(2)

,sample,patient,dataset,disease,cell_type,batch
AAACCTGAGTGGGCTA-1-MS19270-CSF,GSM4104122,MS19270,GSE138266,MS,CSF,0
AAACCTGAGTGTTAGA-1-MS19270-CSF,GSM4104122,MS19270,GSE138266,MS,CSF,0


In [8]:
GSE138266_labels = pd.read_csv(GSE138266_LABELS_PATH, index_col=0)
GSE138266_labels.head(2)

,cell_types_labels
AAACCTGAGTGGGCTA-1-MS19270-CSF-1-0-0-0-0,T cells
AAACCTGAGTGTTAGA-1-MS19270-CSF-1-0-0-0-0,T cells


In [9]:
labels = []
for label in GSE138266_labels.index:
    # drop the last character of the label until it's not a letter
    while not label[-1].isalpha():
        label = label[:-1]
    labels.append(label)
GSE138266_labels.index = labels

GSE138266_labels.head(2)

,cell_types_labels
AAACCTGAGTGGGCTA-1-MS19270-CSF,T cells
AAACCTGAGTGTTAGA-1-MS19270-CSF,T cells


In [10]:
for label in GSE138266_labels.index:
    if label not in adata_GSE138266.obs.index:
        print(label)

In [11]:
for label in adata_GSE138266.obs.index:
    if label not in GSE138266_labels.index:
        print(label)

In [12]:
def assigne_labels(row):
    row['cell_types_labels'] = GSE138266_labels[GSE138266_labels.index==row.name]['cell_types_labels'].values[0]
    return row

adata_GSE138266.obs = adata_GSE138266.obs.apply(assigne_labels, axis=1)

In [13]:
# drop columns batch
adata_GSE138266.obs.drop(columns=['batch'], inplace=True)

In [14]:
adata_GSE138266.obs

,sample,patient,dataset,disease,cell_type,cell_types_labels
AAACCTGAGTGGGCTA-1-MS19270-CSF,GSM4104122,MS19270,GSE138266,MS,CSF,T cells
AAACCTGAGTGTTAGA-1-MS19270-CSF,GSM4104122,MS19270,GSE138266,MS,CSF,T cells
AAACCTGGTCGCGTGT-1-MS19270-CSF,GSM4104122,MS19270,GSE138266,MS,CSF,Monocytes
AAACCTGGTCTCCACT-1-MS19270-CSF,GSM4104122,MS19270,GSE138266,MS,CSF,T cells
AAACCTGGTTTACTCT-1-MS19270-CSF,GSM4104122,MS19270,GSE138266,MS,CSF,T cells
...,...,...,...,...,...,...
TTTGCGCGTGCAACGA-1-PTC85037-PBMC,GSM4104143,PTC85037,GSE138266,PTC,PBMC,T cells
TTTGCGCTCCGTAGTA-1-PTC85037-PBMC,GSM4104143,PTC85037,GSE138266,PTC,PBMC,T cells
TTTGGTTCACCACGTG-1-PTC85037-PBMC,GSM4104143,PTC85037,GSE138266,PTC,PBMC,ILC
TTTGTCACACTGAAGG-1-PTC85037-PBMC,GSM4104143,PTC85037,GSE138266,PTC,PBMC,B cells


In [15]:
adata_GSE138266

AnnData object with n_obs × n_vars = 68038 × 33694
    obs: 'sample', 'patient', 'dataset', 'disease', 'cell_type', 'cell_types_labels'
    var: 'gene_symbols', 'gene_ids'

### GSE194078

In [16]:
adata_GSE194078 = sc.read(GSE194078_PATH)

adata_GSE194078

AnnData object with n_obs × n_vars = 110565 × 36601
    obs: 'patient', 'patient_id', 'disease', 'batch', 'dataset', 'cell_types'
    var: 'gene_ids', 'feature_types'

In [17]:
adata_GSE194078.obs.head(2)

,patient,patient_id,disease,batch,dataset,cell_types
AAACCCACAAGACCGA-1-PBMC,GSM5827375,CYG,Ab-mediated IDD,0,GSE194078,PBMC
AAACCCACACGGCACT-1-PBMC,GSM5827375,CYG,Ab-mediated IDD,0,GSE194078,PBMC


In [18]:
GSE194078_labels = pd.read_csv(GSE194078_LABELS_PATH, index_col=0)
GSE194078_labels.head(2)

,cell_types_labels
AAACCCACAAGACCGA-1-PBMC-GSE194078,B cells
AAACCCACACGGCACT-1-PBMC-GSE194078,T cells


In [19]:
def assigne_labels_GSE194078(row):
    cell_name = row.name + '-GSE194078'
    row['cell_types_labels'] = GSE194078_labels[GSE194078_labels.index==cell_name]['cell_types_labels'].values[0]
    return row

adata_GSE194078.obs = adata_GSE194078.obs.apply(assigne_labels_GSE194078, axis=1)

In [20]:
# rename columns 'patiens' to 'sample' and 'patiens_id' to 'patient_id'
adata_GSE194078.obs.rename(columns={'patiens': 'sample', 'patiens_id': 'patient_id', 'cell_types': 'cell_type'}, inplace=True)

# drop columns batch
adata_GSE194078.obs.drop(columns=['batch'], inplace=True)

In [21]:
adata_GSE194078.obs

,patient,patient_id,disease,dataset,cell_type,cell_types_labels
AAACCCACAAGACCGA-1-PBMC,GSM5827375,CYG,Ab-mediated IDD,GSE194078,PBMC,B cells
AAACCCACACGGCACT-1-PBMC,GSM5827375,CYG,Ab-mediated IDD,GSE194078,PBMC,T cells
AAACCCATCAGAATAG-1-PBMC,GSM5827375,CYG,Ab-mediated IDD,GSE194078,PBMC,T cells
AAACGAAAGGTAAAGG-1-PBMC,GSM5827375,CYG,Ab-mediated IDD,GSE194078,PBMC,T cells
AAACGAACACAGCCAC-1-PBMC,GSM5827375,CYG,Ab-mediated IDD,GSE194078,PBMC,T cells
...,...,...,...,...,...,...
TTTGGTTGTGTTCAGT-11-CSF,GSM5827385,YYW,MS,GSE194078,CSF,T cells
TTTGGTTTCAAGAAAC-11-CSF,GSM5827385,YYW,MS,GSE194078,CSF,T cells
TTTGTTGGTATACCCA-11-CSF,GSM5827385,YYW,MS,GSE194078,CSF,T cells
TTTGTTGGTTATGACC-11-CSF,GSM5827385,YYW,MS,GSE194078,CSF,ILC


In [22]:
adata_GSE194078

AnnData object with n_obs × n_vars = 110565 × 36601
    obs: 'patient', 'patient_id', 'disease', 'dataset', 'cell_type', 'cell_types_labels'
    var: 'gene_ids', 'feature_types'

### Concatenate

adata_GSE239626 adata_GSE194078

In [ ]:
# FIXME

In [23]:
adata_GSE239626.var.head(2)

,gene_ids,feature_types
MIR1302-2HG,ENSG00000243485,Gene Expression
FAM138A,ENSG00000237613,Gene Expression


In [24]:
adata_GSE194078.var.head(2)

,gene_ids,feature_types
MIR1302-2HG,ENSG00000243485,Gene Expression
FAM138A,ENSG00000237613,Gene Expression


In [39]:
new_genes_ref = adata_GSE138266.var['gene_symbols'][~adata_GSE138266.var['gene_symbols'
    ].isin(adata_GSE239626.var['gene_ids'])]

a = sc.AnnData(np.zeros((adata_GSE239626.X.shape[0], len(new_genes_ref))))
a.var.index = new_genes_ref
a.obs.index = adata_GSE239626.obs.index
a.obs = adata_GSE239626.obs

adata_GSE239626_final = anndata.concat([adata_GSE239626, a], axis=1)
adata_GSE239626_final

AnnData object with n_obs × n_vars = 72317 × 37944

In [42]:
adata_GSE239626_final.obs

""
AAACCCAAGACTGTTC-1
AAACCCAAGGATCATA-1
AAACCCAGTTATTCTC-1
AAACCCATCATGAGGG-1
AAACGAAAGCCAGTAG-1
...
TTTGTTGAGCGTCGAA-20
TTTGTTGAGTGTACCT-20
TTTGTTGCAATTGCAC-20
TTTGTTGCATTCAGCA-20


In [32]:
adata_GSE239626_final.var.head(2)

""
MIR1302-2HG
FAM138A


In [33]:
adata_GSE194078_final

AnnData object with n_obs × n_vars = 0 × 37944

In [34]:
adata_GSE194078_final.var.head(2)

""
MIR1302-2HG
FAM138A


adata_GSE138266

In [35]:
adata_GSE138266.var.head(2)

,gene_symbols,gene_ids
0,ENSG00000243485,RP11-34P13.3
1,ENSG00000237613,FAM138A


In [26]:
import anndata
import scipy.sparse as sp


In [36]:
new_genes_13 = adata_GSE239626.var['gene_ids'][~adata_GSE239626.var['gene_ids'
    ].isin(adata_GSE138266.var['gene_symbols'])]

a = sc.AnnData(sp.csr_matrix((adata_GSE138266.X.shape[0], len(new_genes_13))))

a.var.index = new_genes_13
a.obs.index = adata_GSE138266.obs.index
a.obs.sample = adata_GSE138266.obs.sample

adata_GSE138266_final = anndata.concat([adata_GSE138266, a], axis=1)
adata_GSE138266_final

AnnData object with n_obs × n_vars = 68038 × 37944

In [37]:
adata_GSE138266_final.var.head(2)

""
0
1


In [38]:
adata_GSE239626_final.n_vars, adata_GSE194078_final.n_vars, adata_GSE138266_final.n_vars

(37944, 37944, 37944)

Concat

In [29]:
adata = adata_GSE239626_final
adata = adata.concatenate(adata_GSE138266_final)
adata = adata.concatenate(adata_GSE194078_final)

adata

NameError: name 'adata_GSE239626_final' is not defined

In [ ]:
adata.obs['disease'].unique()

array(['MS', 'PTC', 'PST', 'Ab-mediated IDD', 'Normal', 'YYW'],
      dtype=object)

In [ ]:
adata.obs['MS/HC'] = adata.obs['disease'].apply(lambda x: 'MS' if x=='MS' else 'HC')

### Save new dataset

In [ ]:
adata.write('write/transcrittomic.h5ad')